In [2]:
import rasterio

In [3]:
path = 'output/output_alpha.tif'
with rasterio.open(path) as src:
    meta = src.meta
    width = src.width
    height = src.height
    print(f'Dimension {height}x{width}')

Dimension 3130x3219


In [4]:
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

def intersect_tiffs(tiff1_path, tiff2_path, output_path):
    """
    Encuentra la intersección entre dos archivos TIFF y crea uno nuevo con los datos del segundo TIFF
    recortados al área de intersección del primero.
    
    Args:
        tiff1_path (str): Ruta al primer archivo TIFF (se usará su extensión)
        tiff2_path (str): Ruta al segundo archivo TIFF (se usarán sus datos)
        output_path (str): Ruta donde se guardará el nuevo TIFF
    """
    
    # Abrir ambos archivos TIFF
    with rasterio.open(tiff1_path) as src1:
        with rasterio.open(tiff2_path) as src2:
            # Obtener los bounds (límites) del primer TIFF
            bounds = src1.bounds
            
            # Verificar si los sistemas de coordenadas son diferentes
            if src1.crs != src2.crs:
                print('CRS diferentes')
                # Calcular la transformación necesaria
                transform, width, height = calculate_default_transform(
                    src2.crs, src1.crs, src2.width, src2.height, *src2.bounds)
                
                # Crear un array temporal para reproyectar
                temp_data = np.zeros((height, width), dtype=src2.dtypes[0])
                
                # Reproyectar los datos del segundo TIFF al CR.S del primero
                reproject(
                    source=rasterio.band(src2, 1),
                    destination=temp_data,
                    src_transform=src2.transform,
                    src_crs=src2.crs,
                    dst_transform=transform,
                    dst_crs=src1.crs,
                    resampling=Resampling.nearest
                )
                
                # Actualizar los metadatos para el archivo reproyectado
                profile = src2.profile.copy()
                profile.update({
                    'crs': src1.crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })
                
                # Crear un dataset temporal con los datos reproyectados
                with rasterio.MemoryFile() as memfile:
                    with memfile.open(**profile) as temp_dataset:
                        temp_dataset.write(temp_data, 1)
                        # Realizar el recorte usando los bounds del primer TIFF
                        out_image, out_transform = mask(temp_dataset, 
                                                      [{'type': 'Polygon', 
                                                        'coordinates': [[
                                                            [bounds.left, bounds.bottom],
                                                            [bounds.left, bounds.top],
                                                            [bounds.right, bounds.top],
                                                            [bounds.right, bounds.bottom],
                                                            [bounds.left, bounds.bottom]
                                                        ]]}],
                                                      crop=True)
            else:
                # Si tienen el mismo CRS, hacer el recorte directamente
                print('CRS iguales')
                out_image, out_transform = mask(src2, 
                                              [{'type': 'Polygon', 
                                                'coordinates': [[
                                                    [bounds.left, bounds.bottom],
                                                    [bounds.left, bounds.top],
                                                    [bounds.right, bounds.top],
                                                    [bounds.right, bounds.bottom],
                                                    [bounds.left, bounds.bottom]
                                                ]]}],
                                              crop=True)
            
            # Preparar el perfil para el archivo de salida
            out_profile = src2.profile.copy()
            out_profile.update({
                'height': out_image.shape[1],
                'width': out_image.shape[2],
                'transform': out_transform
            })
            
            # Guardar el resultado
            with rasterio.open(output_path, 'w', **out_profile) as dest:
                dest.write(out_image)

In [7]:
path_slope = 'output/output_alpha.tif'
path_rosetta = 'Slope_SRTM_Zone_WGS84.tif'
path_name_output = 'output/prueba.tif'
intersect_tiffs(path_slope, path_rosetta, path_name_output)

CRS iguales
